## Deboost Bubbles

In [ ]:
cd '/home/dpirvu/project/paper_prefactor/'

In [2]:
import os,sys
sys.path.append('/home/dpirvu/python_stuff/')
sys.path.append('/home/dpirvu/project/paper_prefactor/bubble_codes/')
#sys.path.append('/home/dpirvu/project/paper_prefactor/bubble_codes_older/')
sys.path.remove('/home/dpirvu/DarkPhotonxunWISE/hmvec-master')
print(sys.path)
from plotting import *
from bubble_tools import *
from experiment import *
from celluloid import Camera

%matplotlib inline

['/home/dpirvu/project/paper_prefactor', '/cm/shared/apps/python/python37/lib/python37.zip', '/cm/shared/apps/python/python37/lib/python3.7', '/cm/shared/apps/python/python37/lib/python3.7/lib-dynload', '', '/home/dpirvu/.local/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages/IPython/extensions', '/home/dpirvu/.ipython', '/home/dpirvu/python_stuff/', '/home/dpirvu/project/paper_prefactor/bubble_codes/']
['/home/dpirvu/project/paper_prefactor', '/cm/shared/apps/python/python37/lib/python37.zip', '/cm/shared/apps/python/python37/lib/python3.7', '/cm/shared/apps/python/python37/lib/python3.7/lib-dynload', '', '/home/dpirvu/.local/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages/IPython/extensions', '/home/dpirvu/.ipython', '/home/dpirvu/python_stuff/', '/home/dpirvu/project/paper_pr

In [ ]:
def plot_zoomin(bubble, threshold=2., winsize=100, title=None):
    real = np.copy(bubble)
    real = real[0]
    nT, nN = np.shape(real)
    t_centre, x_centre = find_nucleation_center(real, phieq, crit_thresh, crit_rad)
    tl_stop, tr_stop = int(max(0, t_centre - winsize)), int(min(nT, t_centre + winsize//2))
    xl_stop, xr_stop = int(max(0, x_centre - winsize)), int(min(nN, x_centre + winsize))
    real = real[tl_stop:tr_stop, xl_stop:xr_stop]
    nT, nN = np.shape(real)
    tcen, xcen = find_nucleation_center(real, phieq, crit_thresh, crit_rad)
    t, x = np.linspace(-tcen, nT-1-tcen, nT), np.linspace(-xcen, nN-1-xcen, nN)
    real[np.abs(real) > threshold] = threshold
    simple_imshow([real], x, t, title=title, contour=False, ret=False)
    return

def simple_imshow_continue(ax, bubble, xList, tList, vmin, vmax, contour=False, title=None, aspect='auto', cmap='tab20c'):
    ext = [xList[0],xList[-1],tList[0],tList[-1]]
    im = ax.imshow(bubble, interpolation='none', extent=ext, vmin=vmin, vmax=vmax, aspect=aspect, origin='lower', cmap=cmap)
    if contour:
        ax.contour(xList, tList, bubble, levels=5, linewidths=0.5, colors='k')
    clb = plt.colorbar(im, ax = ax, shrink=0.5)
    ax.plot(0, 0, 'bo', ms=3)
    ax.set_title(title)
    return ax


In [3]:
case = 'minus'
general = get_general_model(case)
tempList, massq, right_Vmax, V, dV, Vinv, nTimeMAX, minSim, maxSim = general

tmp = 2

maxSim = (1000 if tmp == 0 else 2000)
temp, m2, sigmafld = get_model(*general, tmp)
exp_params = [nLat, m2, temp]
print('Experiment', exp_params)

Experiment [2048, 0.7, 0.2]


In [5]:
crit_thresh = right_Vmax + 5.*sigmafld; print(crit_thresh)
peak_thresh = 2.8
tv_thresh   = 2.5
crit_rad    = 20

threshm, threshM = right_Vmax + 0.*sigmafld, right_Vmax + 4.*sigmafld
ampList = np.linspace(threshm, threshM, 12); print([round(ii, 2) for ii in ampList])

winsize = 200
plots= False

2.70793613122138
[1.0, 1.12, 1.25, 1.37, 1.5, 1.62, 1.75, 1.87, 1.99, 2.12, 2.24, 2.37]


In [7]:
sim = 10

path2cleansim = clean_sim_location(*exp_params, sim)
if os.path.exists(path2cleansim):
    print('Starting simulation, temp:', sim, temp)

fullreal = np.load(path2cleansim)
nCfull, nTfull, nNfull = np.shape(fullreal)

bubble = np.copy(fullreal[0])
bubble = np.abs(bubble)
bubble = np.array([gaussian_filter(bubble, 1., mode='nearest')])

nC, nT, nN = np.shape(bubble)
tcen, xcen = find_nucleation_center(bubble[0], phieq, crit_thresh, crit_rad)
tList = np.linspace(-tcen, nT-1-tcen, nT)
xList = np.linspace(-xcen, nN-1-xcen, nN)

bool, vellist = True, []

vCOMmeas, varmeas = find_COM_vel(bubble, ampList, winsize, nLat, lightc, phieq, tv_thresh, crit_thresh, crit_rad, plots)
if np.isnan(vCOMmeas):
    print('Dead end at step 0.')
    bool = False

st = 0
doloop = False

if doloop:
    while np.abs(vCOMmeas) >= 0.03 and bool:
        if len(vellist) > 0:
            copy = np.copy(fullreal)[:1]

            wcop = get_totvel_from_list(vellist)
            x, t, copy = boost_bubble(copy, tcen, xcen, wcop)
            vTest, _ = find_COM_vel(copy, ampList, winsize, nLat, lightc, phieq, tv_thresh, crit_thresh, crit_rad, False)
            print('Upon testing this value, we re-measured vTest compared to vCOMmeas', vTest, vCOMmeas)
            if np.abs(vTest) < 0.03:
                vCOMmeas = vTest
                break
            if np.abs(vTest) > np.abs(vellist[-1]):
                vCOMmeas = np.sign(vCOMmeas) * random.randint(5, 10)/10.

        st+=1
        vellist.append(vCOMmeas); print('Step', st, 'vels collected:', vellist);

        t, x, bubble = boost_bubble(bubble, tcen, xcen, vCOMmeas)

        vCOMmeas, varmeas = find_COM_vel(bubble, ampList, winsize, nLat, lightc, phieq, tv_thresh, crit_thresh, crit_rad, plots)
        if np.isnan(vCOMmeas):
            print('Dead end.')
            bool = False

if bool:
    print('Final step.')
    vellist.append(vCOMmeas)

    totvel = get_totvel_from_list(vellist)

    if np.sign(np.mean(test)) < 0:
        print(sim, "changing sign")
        fullreal = -fullreal

    plot_zoomin(np.copy(fullreal), threshold=peak_thresh, winsize=winsize, title='Original zoom')
    
    t, x, fullreal2save = boost_bubble(np.copy(fullreal), tcen, xcen, totvel)
    plot_zoomin(fullreal2save, threshold=peak_thresh, winsize=winsize, title='Final zoom')

    path2RESTsim = rest_sim_location(*exp_params, sim)
    np.save(path2RESTsim, np.array([sim, fullreal2save, totvel]))
    print('Saved sim', sim, 'total final velocity, vellist:', totvel, vellist)


[   0    2    6 ... 1994 1996 1997]


In [8]:
path2RESTsim  = rest_sim_location(*exp_params, sim)
path2cleansim = clean_sim_location(*exp_params, sim)

win = 200

if os.path.exists(path2cleansim) and os.path.exists(path2RESTsim):
    original = np.load(path2cleansim)
    sim, final, totvel = np.load(path2RESTsim, allow_pickle=True)
    print('Sim ', sim, 'total boost', totvel)

    fig, ax = plt.subplots(1, 2, figsize = (8,3))
    for ii, bubble in enumerate([original, final]):
        real = np.copy(bubble[0])
        nT, nN = np.shape(real)
        
        real = gaussian_filter(real, 1, mode='nearest')
        real[np.abs(real) > crit_thresh] = crit_thresh

        tcen, xcen = find_nucleation_center(real, phieq, crit_thresh, crit_rad)
        tl_stop, tr_stop = int(max(0, tcen - win)), int(min(nT, tcen + win//2))
        xl_stop, xr_stop = int(max(0, xcen - win)), int(min(nN, xcen + win))

        real = real[tl_stop:tr_stop, xl_stop:xr_stop]
        nT, nN = np.shape(real)

        tcen, xcen = find_nucleation_center(real, phieq, crit_thresh, crit_rad)
        t, x = np.linspace(-tcen, nT-1-tcen, nT), np.linspace(-xcen, nN-1-xcen, nN)

        if ii == 0: vmin, vmax = np.amin(real), np.amax(real)
        simple_imshow_continue(ax[ii], real, x, t, vmin, vmax, contour=False)

    beautify(ax, times=-70)
    ax[1].grid(which='major', color='black', alpha=1)
    ax[0].grid(which='major', color='black', alpha=1)
    plt.tight_layout()
    plt.show()


[   0    2    6 ... 1994 1996 1997]
